In [12]:
import torch
from torch import nn
from torch import Tensor
import requests
import os
from pathlib import Path
from typing import Callable
from functools import partial


URL = 'https://download.pytorch.org/models/resnet18-5c106cde.pth'

In [13]:
%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
??nn.Sequential

In [22]:
class Residual(nn.Sequential):
    def __init__(self, block: nn.Module,
                 res_func: Callable[[Tensor], Tensor] = None,
                 shortcut: nn.Module = None, *args, **kwargs):

        super().__init__(block)
        self.shortcut = shortcut
        self.res_func = res_func

    def forward(self, x: Tensor) -> Tensor:
        res = x
        if self.shortcut is not None:
            res = self.shortcut(res)
        x = super().forward(x)
        if self.res_func is not None:
            x = self.res_func(x, res)
        return x

def add(x: Tensor, res: Tensor) ->  Tensor:
    return x.add_(res)

ResidualAdd = partial(Residual, res_func=add)

In [25]:
x = torch.rand(1, 32, 48, 48)
block = nn.Conv2d(32, 32, kernel_size=3, padding=1)
adder = ResidualAdd(block)
# 1 + 1
adder(x).shape

adder

Residual(
  (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)

In [37]:
x = torch.tensor(1)
add_one = lambda x: x + 1
adder = ResidualAdd(nn.Identity())
# 1 + 1
assert adder(x) == 2
x = torch.tensor(1)
adder = ResidualAdd(nn.Identity(), shortcut=add_one)
# 1 + 1 + 1

print(adder)
assert adder(x) == 3

<function <lambda> at 0x12fd8a710>


In [47]:
??nn.Sequential

In [46]:
from glasses.nn.models.classification.resnet import ResNetBasicBlock

x = torch.rand(1, 32, 48, 48)
block = ResNetBasicBlock(32, 64)
print(block)
block(x)

ResNetBasicBlock(
  (block): Residual(
    (conv1): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act1): ReLU(inplace=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (shortcut): ResNetShorcut(
      (conv): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (act): ReLU(inplace=True)
)
adsdsdsadasdassdaadsads
shorcut shape torch.Size([1, 32, 48, 48])
torch.Size([1, 64, 48, 48])
torch.Size([1, 32, 48, 48])
shorcut shape torch.Size([1, 64, 48, 48])


RuntimeError: Given groups=1, weight of size [64, 32, 1, 1], expected input[1, 64, 48, 48] to have 32 channels, but got 64 channels instead